# OPTION 1: Predicting Diabetes 

Per [the Mayo Clinic](https://www.mayoclinic.org/diseases-conditions/diabetes/diagnosis-treatment/drc-20371451#:~:text=A%20fasting%20blood%20sugar%20level,separate%20tests%2C%20you%20have%20diabetes.), the Glycated Hemoglobin (A1C) test indicates a person's average blood sugar level for the past two to three months. It measures the percentage of blood sugar attached to hemoglobin, the oxygen-carrying protein in red blood cells. This test is a blood test which does not require fasting.

The higher your blood sugar levels, the more hemoglobin you'll have with sugar attached. 
* A1C level of 6.5% or higher on two separate tests indicates that you have diabetes
* A1C between 5.7 and 6.4 % indicates prediabetes
* A1C below 5.7 is considered normal

# OPTION 2: Predicting Insurance Coverage

# Divider

In [144]:
import pandas as pd
pd.set_option('display.max_columns', 0)
import numpy as np

# styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

## Data Source

### National Health and Nutrition Examination Survey (NHANES)

The [National Health and Nutrition Examination Survey (NHANES)](https://www.cdc.gov/nchs/nhanes/about_nhanes.htm) is a program of continuous studies designed to assess the health and nutritional status of adults and children in the United States. The survey examines a nationally representative sample of about 5,000 persons located across the country each year. The survey is unique in that it combines interviews and physical examinations. The NHANES interview includes demographic, socioeconomic, dietary, and health-related questions. The examination component consists of medical, dental, and physiological measurements, as well as laboratory tests administered by highly trained medical personnel.

NHANES is a major program of the National Center for Health Statistics (NCHS). NCHS is part of the Centers for Disease Control and Prevention (CDC) and has the responsibility for producing vital and health statistics for the Nation.

### Subset for Predicting Diabetes

While NHANES collects a wealth of demographic, laboratory, and medical data, this analysis and predictive model uses a subset comprised of:

* Demographics - collected by trained interviewers using Computer-Assisted Personal Interview (CAPI) system in either English or Spanish, sometimes with assistance from an interpreter. Individuals 16 years and older and emancipated minors were interviewed directly; a proxy provided information for survey participants who were under 16 and for participants who could not answer the questions themselves.

* Body Measures - measured by trained health technicians in the Mobile Examination Center (MEC). 

* Smoking Survey

* Physical Activity Survey

* Blood Pressure

* Total Cholesterol

* A1C

* Insulin


In [145]:
demographic = pd.read_sas('./data/NHANES2017-2018_demographic.xpt')
insurance = pd.read_sas('./data/NHANES2017-2018_insurance.xpt')
measures = pd.read_sas('./data/NHANES2017-2018_body_measures.xpt')

bp = pd.read_sas('./data/NHANES2017-2018_blood_pressure_oscillometric.xpt')
a1c = pd.read_sas('./data/NHANES2017-2018_a1c.xpt')
chol_total = pd.read_sas('./data/NHANES2017-2018_total_cholesterol.xpt')
chol_hdl = pd.read_sas('./data/NHANES2017-2018_hdl_cholesterol.xpt')
chol_ldl = pd.read_sas('./data/NHANES2017-2018_ldl_cholesterol.xpt')
insulin = pd.read_sas('./data/NHANES2017-2018_insulin.xpt')

activity = pd.read_sas('./data/NHANES2017-2018_physical_activity.xpt')
smoking = pd.read_sas('./data/NHANES2017-2018_smoking.xpt')




In [147]:
data = [demographic, insurance, measures, bp, a1c, chol_total, chol_hdl, chol_ldl, 
        insulin, activity, smoking]

In [148]:
for f in data[:-1]:
    f.SEQN = f.SEQN.map(lambda x: int(x))
    f.set_index('SEQN', inplace=True)
    display(f.info())
    print('_____'*20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9254 entries, 93703 to 102956
Data columns (total 45 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SDDSRVYR  9254 non-null   float64
 1   RIDSTATR  9254 non-null   float64
 2   RIAGENDR  9254 non-null   float64
 3   RIDAGEYR  9254 non-null   float64
 4   RIDAGEMN  597 non-null    float64
 5   RIDRETH1  9254 non-null   float64
 6   RIDRETH3  9254 non-null   float64
 7   RIDEXMON  8704 non-null   float64
 8   RIDEXAGM  3433 non-null   float64
 9   DMQMILIZ  6004 non-null   float64
 10  DMQADFC   561 non-null    float64
 11  DMDBORN4  9254 non-null   float64
 12  DMDCITZN  9251 non-null   float64
 13  DMDYRSUS  1948 non-null   float64
 14  DMDEDUC3  2306 non-null   float64
 15  DMDEDUC2  5569 non-null   float64
 16  DMDMARTL  5569 non-null   float64
 17  RIDEXPRG  1110 non-null   float64
 18  SIALANG   9254 non-null   float64
 19  SIAPROXY  9254 non-null   float64
 20  SIAINTRP  9254 non-null 

None

____________________________________________________________________________________________________
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6401 entries, 93705 to 102956
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   LBXGH   6045 non-null   float64
dtypes: float64(1)
memory usage: 100.0 KB


None

____________________________________________________________________________________________________
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7132 entries, 93705 to 102956
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   BPAOARM   7132 non-null   object 
 1   BPAOCSZ   6144 non-null   float64
 2   BPAOMNTS  6144 non-null   float64
 3   BPXOSY1   6143 non-null   float64
 4   BPXODI1   6143 non-null   float64
 5   BPXOSY2   6123 non-null   float64
 6   BPXODI2   6123 non-null   float64
 7   BPXOSY3   6094 non-null   float64
 8   BPXODI3   6094 non-null   float64
 9   BPXOPLS1  5262 non-null   float64
 10  BPXOPLS2  5244 non-null   float64
 11  BPXOPLS3  5220 non-null   float64
dtypes: float64(11), object(1)
memory usage: 724.3+ KB


...

____________________________________________________________________________________________________
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3036 entries, 93708 to 102956
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   WTSAF2YR  3036 non-null   float64
 1   LBXTR     2834 non-null   float64
 2   LBDTRSI   2834 non-null   float64
 3   LBDLDL    2808 non-null   float64
 4   LBDLDLSI  2808 non-null   float64
 5   LBDLDLM   2808 non-null   float64
 6   LBDLDMSI  2808 non-null   float64
 7   LBDLDLN   2827 non-null   float64
 8   LBDLDNSI  2827 non-null   float64
dtypes: float64(9)
memory usage: 237.2 KB


None

____________________________________________________________________________________________________
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6724 entries, 93705 to 102956
Data columns (total 36 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SMQ020    5856 non-null   float64
 1   SMD030    2359 non-null   float64
 2   SMQ040    2359 non-null   float64
 3   SMQ050Q   1338 non-null   float64
 4   SMQ050U   1255 non-null   float64
 5   SMD057    1338 non-null   float64
 6   SMQ078    793 non-null    float64
 7   SMD641    1063 non-null   float64
 8   SMD650    1022 non-null   float64
 9   SMD093    1021 non-null   float64
 10  SMDUPCA   6724 non-null   object 
 11  SMD100BR  6724 non-null   object 
 12  SMD100FL  929 non-null    float64
 13  SMD100MN  929 non-null    float64
 14  SMD100LN  929 non-null    float64
 15  SMD100TR  695 non-null    float64
 16  SMD100NI  695 non-null    float64
 17  SMD100CO  695 non-null    float64
 18  S

None

____________________________________________________________________________________________________


#### Demographics 

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.htm

Comprised of individual, family, and household-level information. 

In [150]:
demographic.describe().round(1)

,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,DMQADFC,DMDBORN4,DMDCITZN,DMDYRSUS,DMDEDUC3,DMDEDUC2,DMDMARTL,RIDEXPRG,SIALANG,SIAPROXY,SIAINTRP,FIALANG,FIAPROXY,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANGA,DMDHHSIZ,DMDFMSIZ,DMDHHSZA,DMDHHSZB,DMDHHSZE,DMDHRGND,DMDHRAGZ,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
count,9254.0,9254.0,9254.0,9254.0,597.0,9254.0,9254.0,8704.0,3433.0,6004.0,561.0,9254.0,9251.0,1948.0,2306.0,5569.0,5569.0,1110.0,9254.0,9254.0,9254.0,8780.0,8780.0,8780.0,6684.0,6684.0,6684.0,4977.0,9254.0,9254.0,9254.0,9254.0,9254.0,9254.0,9254.0,8764.0,9063.0,4751.0,9254.0,9254.0,9254.0,9254.0,8763.0,8780.0,8023.0
mean,10.0,1.9,1.5,34.3,10.4,3.2,3.5,1.5,107.5,1.9,1.5,1.2,1.1,9.3,6.3,3.5,2.7,2.0,1.1,1.7,2.0,1.1,2.0,2.0,1.1,2.0,2.0,1.1,3.7,3.6,0.5,0.9,0.5,1.5,2.9,2.1,1.5,2.1,34670.7,34670.7,1.5,141.0,12.5,12.2,2.4
std,0.0,0.2,0.5,25.5,7.1,1.3,1.7,0.5,70.6,0.3,0.6,1.6,0.5,18.6,5.8,1.2,3.1,0.4,0.3,0.5,0.2,0.3,0.0,0.2,0.3,0.1,0.1,0.4,1.7,1.8,0.8,1.1,0.8,0.5,0.8,0.7,0.7,0.7,41356.7,43344.0,0.5,4.2,17.3,17.2,1.6
min,10.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2571.1,0.0,1.0,134.0,1.0,1.0,0.0
25%,10.0,2.0,1.0,11.0,4.0,3.0,3.0,1.0,43.0,2.0,1.0,1.0,1.0,3.0,3.0,3.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,2.0,2.0,1.0,2.0,13074.4,12347.3,1.0,137.0,6.0,6.0,1.0
50%,10.0,2.0,2.0,31.0,10.0,3.0,3.0,2.0,106.0,2.0,1.0,1.0,1.0,6.0,6.0,4.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,4.0,4.0,0.0,0.0,0.0,1.0,3.0,2.0,1.0,2.0,21098.5,21059.9,2.0,141.0,8.0,8.0,1.9
75%,10.0,2.0,2.0,58.0,17.0,4.0,4.0,2.0,166.0,2.0,2.0,1.0,1.0,7.0,9.0,4.0,5.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,2.0,1.0,2.0,4.0,2.0,2.0,3.0,36923.3,37562.0,2.0,145.0,14.0,14.0,3.7
max,10.0,2.0,2.0,80.0,24.0,5.0,7.0,2.0,239.0,9.0,7.0,99.0,9.0,99.0,66.0,9.0,77.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,7.0,7.0,3.0,3.0,3.0,2.0,4.0,3.0,3.0,3.0,433085.0,419762.8,2.0,148.0,99.0,99.0,5.0


In [151]:
demographic.columns

Index(['SDDSRVYR', 'RIDSTATR', 'RIAGENDR', 'RIDAGEYR', 'RIDAGEMN', 'RIDRETH1',
       'RIDRETH3', 'RIDEXMON', 'RIDEXAGM', 'DMQMILIZ', 'DMQADFC', 'DMDBORN4',
       'DMDCITZN', 'DMDYRSUS', 'DMDEDUC3', 'DMDEDUC2', 'DMDMARTL', 'RIDEXPRG',
       'SIALANG', 'SIAPROXY', 'SIAINTRP', 'FIALANG', 'FIAPROXY', 'FIAINTRP',
       'MIALANG', 'MIAPROXY', 'MIAINTRP', 'AIALANGA', 'DMDHHSIZ', 'DMDFMSIZ',
       'DMDHHSZA', 'DMDHHSZB', 'DMDHHSZE', 'DMDHRGND', 'DMDHRAGZ', 'DMDHREDZ',
       'DMDHRMAZ', 'DMDHSEDZ', 'WTINT2YR', 'WTMEC2YR', 'SDMVPSU', 'SDMVSTRA',
       'INDHHIN2', 'INDFMIN2', 'INDFMPIR'],
      dtype='object')

In [152]:
keepcols_demographic = ['RIAGENDR', 'RIDAGEYR', 'RIDRETH3', 'DMQMILIZ', 'DMDBORN4', 'DMDCITZN',
                    'DMDEDUC2', 'DMDMARTL', 'RIDEXPRG', 'INDHHIN2', 'INDFMIN2', 'INDFMPIR']

In [153]:
keep_demographic = demographic[keepcols_demographic]
keep_demographic.rename(mapper={'RIAGENDR': 'gender',
                                'RIDAGEYR': 'age',
                                'RIDRETH3': 'race',
                                'DMQMILIZ': 'veteran_status',
                                'DMDBORN4': 'country_of_birth',
                                'DMDCITZN': 'citizen_status',
                                'DMDEDUC2': 'education',
                                'DMDMARTL': 'marital_status',
                                'RIDEXPRG': 'pregnancy_status',
                                'INDHHIN2': 'annual_household_income',
                                'INDFMIN2': 'annual_family_income',
                                'INDFMPIR': 'income_poverty_ratio'}, 
                       axis=1, inplace=True)
keep_demographic

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,pregnancy_status,annual_household_income,annual_family_income,income_poverty_ratio
SEQN,,,,,,,,,,,,
93703,2.0,2.0,6.0,NaN,1.0,1.0,NaN,NaN,NaN,15.0,15.0,5.00
93704,1.0,2.0,3.0,NaN,1.0,1.0,NaN,NaN,NaN,15.0,15.0,5.00
93705,2.0,66.0,4.0,2.0,1.0,1.0,2.0,3.0,NaN,3.0,3.0,0.82
93706,1.0,18.0,6.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
93707,1.0,13.0,7.0,NaN,1.0,1.0,NaN,NaN,NaN,10.0,10.0,1.88
...,...,...,...,...,...,...,...,...,...,...,...,...
102952,2.0,70.0,6.0,2.0,2.0,1.0,3.0,1.0,NaN,4.0,4.0,0.95
102953,1.0,42.0,1.0,2.0,2.0,2.0,3.0,4.0,NaN,12.0,12.0,NaN
102954,2.0,41.0,4.0,2.0,1.0,1.0,5.0,5.0,2.0,10.0,10.0,1.18


In [154]:
keep_demographic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9254 entries, 93703 to 102956
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gender                   9254 non-null   float64
 1   age                      9254 non-null   float64
 2   race                     9254 non-null   float64
 3   veteran_status           6004 non-null   float64
 4   country_of_birth         9254 non-null   float64
 5   citizen_status           9251 non-null   float64
 6   education                5569 non-null   float64
 7   marital_status           5569 non-null   float64
 8   pregnancy_status         1110 non-null   float64
 9   annual_household_income  8763 non-null   float64
 10  annual_family_income     8780 non-null   float64
 11  income_poverty_ratio     8023 non-null   float64
dtypes: float64(12)
memory usage: 939.9 KB


Notes:
* Age 
    * Individuals aged 80 and over are topcoded at 80. In NHANES 2017-2018, the weighted mean age for participants 80+ is 85
    * Individuals' ages are reported in months for those 24 months (2 yrs) and younger
* Income-Poverty Ratio - calculated by dividing family (or individual) income by the poverty guidelines specific to the survey year. The value was not computed if the respondent only reported income as < $20,000 or ≥ $20,000. If family income was reported as a more detailed category, the midpoint of the range was used to compute the ratio. Values at or above 5.00 were coded as 5.00 or more because of disclosure concerns. The values were not computed if the income data was missing.


#### Health Insurance

[Source](https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/HIQ_J.htm)

The Health Insurance questionnaire (variable name prefix HIQ) provides respondent-level interview data on insurance coverage, type of insurance coverage, coverage of prescription drugs, and uninsured status during the past 12 months.

In [155]:
insurance

,SEQN,HIQ011,HIQ031A,HIQ031B,HIQ031C,HIQ031D,HIQ031E,HIQ031F,HIQ031H,HIQ031I,HIQ031J,HIQ031AA,HIQ260,HIQ105,HIQ270,HIQ210
0,93703.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
1,93704.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
2,93705.0,1.0,NaN,15.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0
3,93706.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
4,93707.0,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9249,102952.0,1.0,14.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0
9250,102953.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
9251,102954.0,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
9252,102955.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0


In [156]:
insurance.columns

Index(['SEQN', 'HIQ011', 'HIQ031A', 'HIQ031B', 'HIQ031C', 'HIQ031D', 'HIQ031E',
       'HIQ031F', 'HIQ031H', 'HIQ031I', 'HIQ031J', 'HIQ031AA', 'HIQ260',
       'HIQ105', 'HIQ270', 'HIQ210'],
      dtype='object')

In [157]:
keepcols_insurance = ['HIQ011', 'HIQ031A', 'HIQ031B', 'HIQ031C', 'HIQ031D', 'HIQ031E',
       'HIQ031F', 'HIQ031H', 'HIQ031I', 'HIQ031J', 'HIQ031AA', 'HIQ270', 'HIQ210']

mapper_insurance = {'HIQ011': 'coverage_status',
          'HIQ031A': 'covered_private',
          'HIQ031B': 'covered_medicare',
          'HIQ031C': 'covered_medigap',
          'HIQ031D': 'covered_medicaid',
          'HIQ031E': 'covered_chip',
          'HIQ031F': 'covered_military',
          'HIQ031H': 'covered_state',
          'HIQ031I': 'covered_other_gov',
          'HIQ031J': 'covered_single_service',
          'HIQ031AA': 'not_covered',
          'HIQ270': 'prescription_coverage',
          'HIQ210': 'uninsured_in_last_year'}

In [158]:
keep_insurance = insurance[keepcols_insurance]
keep_insurance.rename(mapper=mapper_insurance, axis=1, inplace=True)
keep_insurance

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,not_covered,prescription_coverage,uninsured_in_last_year
0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
1,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
2,1.0,NaN,15.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
3,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
4,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9249,1.0,14.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
9250,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
9251,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
9252,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0


In [159]:
display(keep_insurance.coverage_status.value_counts())
display(keep_insurance.coverage_status.value_counts(normalize=True))

1.0    8157
2.0    1072
9.0      18
7.0       7
Name: coverage_status, dtype: int64

1.0    0.881457
2.0    0.115842
9.0    0.001945
7.0    0.000756
Name: coverage_status, dtype: float64

In [160]:
display(keep_insurance.uninsured_in_last_year.value_counts())
display(keep_insurance.uninsured_in_last_year.value_counts(normalize=True))

2.0    7591
1.0     561
9.0      18
7.0       1
Name: uninsured_in_last_year, dtype: int64

2.0    0.929017
1.0    0.068657
9.0    0.002203
7.0    0.000122
Name: uninsured_in_last_year, dtype: float64

Coverage status values:
* 1.0 - Yes
* 2.0 - No
* 7.0 - Refused
* 9.0 - Don't know

#### Body Measures

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BMX_J.htm

NHANES body measures data are used to monitor trends in infant and child growth, to estimate the prevalence of overweight and obesity in U.S. children, adolescents, and adults, and to examine the associations between body weight and the health and nutritional status of the U.S. population.

All survey participants were eligible for the body measures component. There were no medical, safety, or other exclusions for the body measurements protocol. The health technicians used their discretion to obtain as many measures as practical for persons who used a wheelchair.

In [161]:
measures

,BMDSTATS,BMXWT,BMIWT,BMXRECUM,BMIRECUM,BMXHEAD,BMIHEAD,BMXHT,BMIHT,BMXBMI,BMXLEG,BMILEG,BMXARML,BMIARML,BMXARMC,BMIARMC,BMXWAIST,BMIWAIST,BMXHIP,BMIHIP
SEQN,,,,,,,,,,,,,,,,,,,,
93703,1.0,13.7,3.0,89.6,NaN,NaN,NaN,88.6,NaN,17.5,NaN,NaN,18.0,NaN,16.2,NaN,48.2,NaN,NaN,NaN
93704,1.0,13.9,NaN,95.0,NaN,NaN,NaN,94.2,NaN,15.7,NaN,NaN,18.6,NaN,15.2,NaN,50.0,NaN,NaN,NaN
93705,1.0,79.5,NaN,NaN,NaN,NaN,NaN,158.3,NaN,31.7,37.0,NaN,36.0,NaN,32.0,NaN,101.8,NaN,110.0,NaN
93706,1.0,66.3,NaN,NaN,NaN,NaN,NaN,175.7,NaN,21.5,46.6,NaN,38.8,NaN,27.0,NaN,79.3,NaN,94.4,NaN
93707,1.0,45.4,NaN,NaN,NaN,NaN,NaN,158.4,NaN,18.1,38.1,NaN,33.8,NaN,21.5,NaN,64.1,NaN,83.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102952,1.0,49.0,NaN,NaN,NaN,NaN,NaN,156.5,NaN,20.0,34.4,NaN,32.6,NaN,25.1,NaN,82.2,NaN,87.3,NaN
102953,1.0,97.4,NaN,NaN,NaN,NaN,NaN,164.9,NaN,35.8,38.2,NaN,36.6,NaN,40.6,NaN,114.8,NaN,112.8,NaN
102954,1.0,69.1,NaN,NaN,NaN,NaN,NaN,162.6,NaN,26.1,39.2,NaN,35.2,NaN,26.8,NaN,86.4,NaN,102.7,NaN


In [162]:
measures.columns

Index(['BMDSTATS', 'BMXWT', 'BMIWT', 'BMXRECUM', 'BMIRECUM', 'BMXHEAD',
       'BMIHEAD', 'BMXHT', 'BMIHT', 'BMXBMI', 'BMXLEG', 'BMILEG', 'BMXARML',
       'BMIARML', 'BMXARMC', 'BMIARMC', 'BMXWAIST', 'BMIWAIST', 'BMXHIP',
       'BMIHIP'],
      dtype='object')

In [163]:
keepcols_measures = ['BMXWT', 'BMXHT', 'BMXBMI', 'BMXWAIST', 'BMXHIP']

In [164]:
keep_measures = measures[keepcols_measures]
keep_measures.rename(mapper={'BMXWT': 'weight_kg', 
                             'BMXHT': 'height_cm',
                             'BMXBMI': 'BMI',
                             'BMXWAIST': 'waist_circumference_cm',
                             'BMXHIP': 'hip_circumference_cm'},
                    axis=1, inplace=True)
keep_measures

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm
SEQN,,,,,
93703,13.7,88.6,17.5,48.2,NaN
93704,13.9,94.2,15.7,50.0,NaN
93705,79.5,158.3,31.7,101.8,110.0
93706,66.3,175.7,21.5,79.3,94.4
93707,45.4,158.4,18.1,64.1,83.0
...,...,...,...,...,...
102952,49.0,156.5,20.0,82.2,87.3
102953,97.4,164.9,35.8,114.8,112.8
102954,69.1,162.6,26.1,86.4,102.7


#### Blood Pressure

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BPXO_J.htm

Blood pressure measures used in this analysis were taken using a digital upper-arm electronic blood pressure measurement device. 3 measurements were taken 60 seconds apart after the individual had been resting quietly in a seated position for 5 minutes. For the purposes of this analysis, I have calculated the average systolic and average diastolic measurements across the 3 readings.

In [165]:
bp

,BPAOARM,BPAOCSZ,BPAOMNTS,BPXOSY1,BPXODI1,BPXOSY2,BPXODI2,BPXOSY3,BPXODI3,BPXOPLS1,BPXOPLS2,BPXOPLS3
SEQN,,,,,,,,,,,,
93705,b'R',4.0,-20.0,164.0,66.0,165.0,66.0,172.0,66.0,52.0,51.0,49.0
93706,b'R',3.0,138.0,126.0,74.0,128.0,68.0,133.0,71.0,76.0,83.0,73.0
93707,b'R',2.0,12.0,136.0,71.0,133.0,72.0,139.0,71.0,100.0,89.0,91.0
93708,b'R',3.0,22.0,146.0,82.0,142.0,76.0,151.0,81.0,67.0,65.0,71.0
93709,b'R',4.0,58.0,120.0,83.0,124.0,81.0,113.0,74.0,64.0,62.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...
102952,b'R',3.0,97.0,154.0,92.0,144.0,84.0,145.0,69.0,88.0,84.0,74.0
102953,b'R',4.0,-57.0,135.0,91.0,133.0,86.0,132.0,83.0,76.0,79.0,78.0
102954,b'',3.0,-101.0,123.0,75.0,119.0,71.0,122.0,73.0,NaN,NaN,NaN


In [166]:
bp.columns

Index(['BPAOARM', 'BPAOCSZ', 'BPAOMNTS', 'BPXOSY1', 'BPXODI1', 'BPXOSY2',
       'BPXODI2', 'BPXOSY3', 'BPXODI3', 'BPXOPLS1', 'BPXOPLS2', 'BPXOPLS3'],
      dtype='object')

In [167]:
keepcols_bp = ['BPXOSY1', 'BPXODI1', 'BPXOSY2', 'BPXODI2', 'BPXOSY3', 
               'BPXODI3', 'BPXOPLS1', 'BPXOPLS2', 'BPXOPLS3']

mapper_bp = {'BPXOSY1': 'sys_1', 
          'BPXODI1': 'dias_1',
          'BPXOSY2': 'sys_2',
          'BPXODI2': 'dias_2',
          'BPXOSY3': 'sys_3',
          'BPXODI3': 'dias_3',
          'BPXOPLS1': 'pulse_1',
          'BPXOPLS2': 'pulse_2',
          'BPXOPLS3': 'pulse_3'}

In [168]:
keep_bp = bp[keepcols_bp]
keep_bp.rename(mapper=mapper_bp, axis=1, inplace=True)
keep_bp

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,sys_1,dias_1,sys_2,dias_2,sys_3,dias_3,pulse_1,pulse_2,pulse_3
SEQN,,,,,,,,,
93705,164.0,66.0,165.0,66.0,172.0,66.0,52.0,51.0,49.0
93706,126.0,74.0,128.0,68.0,133.0,71.0,76.0,83.0,73.0
93707,136.0,71.0,133.0,72.0,139.0,71.0,100.0,89.0,91.0
93708,146.0,82.0,142.0,76.0,151.0,81.0,67.0,65.0,71.0
93709,120.0,83.0,124.0,81.0,113.0,74.0,64.0,62.0,61.0
...,...,...,...,...,...,...,...,...,...
102952,154.0,92.0,144.0,84.0,145.0,69.0,88.0,84.0,74.0
102953,135.0,91.0,133.0,86.0,132.0,83.0,76.0,79.0,78.0
102954,123.0,75.0,119.0,71.0,122.0,73.0,NaN,NaN,NaN


In [169]:
keep_bp['avg_systolic'] = ((keep_bp['sys_1'] + keep_bp['sys_2'] + keep_bp['sys_3']) / 3).round(1)
keep_bp['avg_diastolic'] = ((keep_bp['dias_1'] + keep_bp['dias_2'] + keep_bp['dias_3']) / 3).round(1)
keep_bp['avg_pulse'] = ((keep_bp['pulse_1'] + keep_bp['pulse_2'] + keep_bp['pulse_3']) / 3).round(1)
keep_bp

<ipython-input-169-9a2353af5978>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keep_bp['avg_systolic'] = ((keep_bp['sys_1'] + keep_bp['sys_2'] + keep_bp['sys_3']) / 3).round(1)
<ipython-input-169-9a2353af5978>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keep_bp['avg_diastolic'] = ((keep_bp['dias_1'] + keep_bp['dias_2'] + keep_bp['dias_3']) / 3).round(1)


,sys_1,dias_1,sys_2,dias_2,sys_3,dias_3,pulse_1,pulse_2,pulse_3,avg_systolic,avg_diastolic,avg_pulse
SEQN,,,,,,,,,,,,
93705,164.0,66.0,165.0,66.0,172.0,66.0,52.0,51.0,49.0,167.0,66.0,50.7
93706,126.0,74.0,128.0,68.0,133.0,71.0,76.0,83.0,73.0,129.0,71.0,77.3
93707,136.0,71.0,133.0,72.0,139.0,71.0,100.0,89.0,91.0,136.0,71.3,93.3
93708,146.0,82.0,142.0,76.0,151.0,81.0,67.0,65.0,71.0,146.3,79.7,67.7
93709,120.0,83.0,124.0,81.0,113.0,74.0,64.0,62.0,61.0,119.0,79.3,62.3
...,...,...,...,...,...,...,...,...,...,...,...,...
102952,154.0,92.0,144.0,84.0,145.0,69.0,88.0,84.0,74.0,147.7,81.7,82.0
102953,135.0,91.0,133.0,86.0,132.0,83.0,76.0,79.0,78.0,133.3,86.7,77.7
102954,123.0,75.0,119.0,71.0,122.0,73.0,NaN,NaN,NaN,121.3,73.0,NaN


In [170]:
keep_bp2 = keep_bp[['avg_systolic', 'avg_diastolic', 'avg_pulse']]
keep_bp2

,avg_systolic,avg_diastolic,avg_pulse
SEQN,,,
93705,167.0,66.0,50.7
93706,129.0,71.0,77.3
93707,136.0,71.3,93.3
93708,146.3,79.7,67.7
93709,119.0,79.3,62.3
...,...,...,...
102952,147.7,81.7,82.0
102953,133.3,86.7,77.7
102954,121.3,73.0,NaN


#### Lab - A1C

In [171]:
a1c

,LBXGH
SEQN,
93705,6.2
93706,5.2
93707,5.6
93708,6.2
93709,6.3
...,...
102952,7.4
102953,5.9
102954,5.2


In [172]:
a1c.rename(mapper={'LBXGH': 'glycohemoglobin'}, axis=1, inplace=True)

In [173]:
def diagnose(a1c_value):
    diagnosis = None
    if a1c_value >= 6.5:
        diagnosis = 'Diabetic'
    if (a1c_value >= 5.7) & (a1c_value < 6.5):
        diagnosis = 'Prediabetic'
    if a1c_value < 5.7:
        diagnosis = 'Normal'
    return diagnosis

In [174]:
a1c['calculated_diagnosis'] = a1c.glycohemoglobin.map(lambda x: diagnose(x))
a1c

,glycohemoglobin,calculated_diagnosis
SEQN,,
93705,6.2,Prediabetic
93706,5.2,Normal
93707,5.6,Normal
93708,6.2,Prediabetic
93709,6.3,Prediabetic
...,...,...
102952,7.4,Diabetic
102953,5.9,Prediabetic
102954,5.2,Normal


In [175]:
display(a1c.calculated_diagnosis.value_counts())
display(a1c.calculated_diagnosis.value_counts(normalize=True).round(2))

Normal         3659
Prediabetic    1635
Diabetic        751
Name: calculated_diagnosis, dtype: int64

Normal         0.61
Prediabetic    0.27
Diabetic       0.12
Name: calculated_diagnosis, dtype: float64

#### Lab - Cholesterol

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/TCHOL_J.htm

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/TRIGLY_J.htm

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/HDL_J.htm

A complete cholesterol test — also called a lipid panel or lipid profile — is a blood test that can measure the amount of cholesterol and triglycerides in your blood. The blood lipids measurements in NHANES include total cholesterol, high-density lipoprotein cholesterol (HDL-C), low-density lipoproteins cholesterol (LDL-C), and triglycerides. 

* **Total cholesterol** - sum of your blood's cholesterol content
    * Less than 200 mg/dL - desirable
    * 200-239 mg/dL - borderline
    * Greater than 240 mg/dL - high
* **High-density lipoprotein (HDL) cholesterol** - the "good" cholesterol because it helps carry away LDL cholesterol, thus keeping arteries open and your blood flowing more freely.
    * Greater than 60 mg/dL - best
    * 40-59 mg/dL - good
    * Less than 50 (women) or 40 (men) - poor
* **Low-density lipoprotein (LDL) cholesterol** "bad" cholesterol; too much of it in your blood causes the buildup of fatty deposits (plaques) in your arteries (atherosclerosis), which reduces blood flow. These plaques sometimes rupture and can lead to a heart attack or stroke.
    * Less than 100 mg/dL - optimal
    * 100-129 mg/dL - high for those with coronary artery disease 
    * 130-159 mg/dL - borderline
    * Greater than 160 mg/dL - high
    * Greater than 190 mg/dL - very high
* **Triglycerides** - a type of fat in the blood created from calories your body doesn't need. High triglyceride levels are associated with several factors, including being overweight, eating too many sweets or drinking too much alcohol, smoking, being sedentary, or having diabetes with elevated blood sugar levels.
    * Less than 150 mg/dL - desirable
    * 150-199 mg/dL - borderline
    * 200-499 mg/dL - high
    * Greater than 500 mg/dL - very high

https://www.mayoclinic.org/tests-procedures/cholesterol-test/about/pac-20384601

In [176]:
chol_total = chol_total.drop(columns='LBDTCSI').rename(mapper={'LBXTC': 'cholesterol_total'}, axis=1)
chol_total

,cholesterol_total
SEQN,
93705,157.0
93706,148.0
93707,189.0
93708,209.0
93709,176.0
...,...
102952,119.0
102953,182.0
102954,172.0


In [177]:
chol_hdl = chol_hdl.drop(columns='LBDHDDSI').rename(mapper={'LBDHDD': 'cholesterol_hdl'}, axis=1)
chol_hdl

,cholesterol_hdl
SEQN,
93705,60.0
93706,47.0
93707,68.0
93708,88.0
93709,65.0
...,...
102952,60.0
102953,49.0
102954,54.0


In [178]:
chol_ldl

,WTSAF2YR,LBXTR,LBDTRSI,LBDLDL,LBDLDLSI,LBDLDLM,LBDLDMSI,LBDLDLN,LBDLDNSI
SEQN,,,,,,,,,
93708,2.565368e+04,58.0,0.655,109.0,2.819,107.0,2.767,111.0,2.870
93711,2.922605e+04,48.0,0.542,156.0,4.034,152.0,3.931,158.0,4.086
93717,1.371510e+05,102.0,1.152,140.0,3.620,138.0,3.569,142.0,3.672
93718,5.888331e+04,46.0,0.519,80.0,2.069,77.0,1.991,79.0,2.043
93719,5.397605e-79,80.0,0.903,35.0,0.905,35.0,0.905,35.0,0.905
...,...,...,...,...,...,...,...,...,...
102947,3.208847e+04,35.0,0.395,66.0,1.707,62.0,1.603,64.0,1.655
102948,2.911334e+05,52.0,0.587,123.0,3.181,119.0,3.077,124.0,3.207
102952,3.273305e+04,78.0,0.881,43.0,1.112,43.0,1.112,43.0,1.112


In [179]:
chol_ldl.describe().round(1)

,WTSAF2YR,LBXTR,LBDTRSI,LBDLDL,LBDLDLSI,LBDLDLM,LBDLDMSI,LBDLDLN,LBDLDNSI
count,3036.0,2834.0,2834.0,2808.0,2808.0,2808.0,2808.0,2827.0,2827.0
mean,89741.9,107.3,1.2,106.9,2.8,107.2,2.8,108.5,2.8
std,109439.5,98.3,1.1,35.6,0.9,35.6,0.9,36.3,0.9
min,0.0,10.0,0.1,18.0,0.5,21.0,0.5,20.0,0.5
25%,29748.3,58.0,0.7,82.0,2.1,82.0,2.1,83.0,2.1
50%,55057.3,87.0,1.0,103.0,2.7,104.0,2.7,105.0,2.7
75%,101741.7,130.0,1.5,128.0,3.3,129.0,3.3,130.0,3.4
max,944153.2,2684.0,30.3,357.0,9.2,358.0,9.3,359.0,9.3


In [180]:
chol_ldl.columns

Index(['WTSAF2YR', 'LBXTR', 'LBDTRSI', 'LBDLDL', 'LBDLDLSI', 'LBDLDLM',
       'LBDLDMSI', 'LBDLDLN', 'LBDLDNSI'],
      dtype='object')

In [181]:
keepcols_chol_ldl = ['LBXTR', 'LBDLDL']
mapper_chol_ldl = {'LBXTR': 'triglyceride', 'LBDLDL': 'cholesterol_ldl'}

In [182]:
keep_chol_ldl = chol_ldl[keepcols_chol_ldl].rename(mapper=mapper_chol_ldl, axis=1)
keep_chol_ldl

,triglyceride,cholesterol_ldl
SEQN,,
93708,58.0,109.0
93711,48.0,156.0
93717,102.0,140.0
93718,46.0,80.0
93719,80.0,35.0
...,...,...
102947,35.0,66.0
102948,52.0,123.0
102952,78.0,43.0


In [183]:
chol1 = pd.merge(keep_chol_ldl, chol_hdl, how='outer', on='SEQN')
chol1

,triglyceride,cholesterol_ldl,cholesterol_hdl
SEQN,,,
93708,58.0,109.0,88.0
93711,48.0,156.0,72.0
93717,102.0,140.0,53.0
93718,46.0,80.0,63.0
93719,80.0,35.0,46.0
...,...,...,...
102944,NaN,NaN,50.0
102946,NaN,NaN,50.0
102949,NaN,NaN,73.0


In [184]:
cholesterol = pd.merge(chol1, chol_total, how='outer', on='SEQN')
cholesterol

,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total
SEQN,,,,
93708,58.0,109.0,88.0,209.0
93711,48.0,156.0,72.0,238.0
93717,102.0,140.0,53.0,213.0
93718,46.0,80.0,63.0,152.0
93719,80.0,35.0,46.0,97.0
...,...,...,...,...
102944,NaN,NaN,50.0,214.0
102946,NaN,NaN,50.0,214.0
102949,NaN,NaN,73.0,201.0


#### Lab - Insulin

In [185]:
insulin

,WTSAF2YR,LBXIN,LBDINSI,LBDINLC
SEQN,,,,
93708,2.565368e+04,9.72,58.32,5.397605e-79
93711,2.922605e+04,5.28,31.68,5.397605e-79
93717,1.371510e+05,3.94,23.64,5.397605e-79
93718,5.888331e+04,4.89,29.34,5.397605e-79
93719,5.397605e-79,10.94,65.64,5.397605e-79
...,...,...,...,...
102947,3.208847e+04,0.71,4.26,1.000000e+00
102948,2.911334e+05,5.42,32.52,5.397605e-79
102952,3.273305e+04,4.39,26.34,5.397605e-79


In [186]:
keep_insulin = pd.DataFrame(insulin['LBXIN'])
keep_insulin.rename(mapper={'LBXIN': 'insulin'}, axis=1, inplace=True)

In [187]:
keep_insulin

,insulin
SEQN,
93708,9.72
93711,5.28
93717,3.94
93718,4.89
93719,10.94
...,...
102947,0.71
102948,5.42
102952,4.39


#### Physical Activity

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/PAQ_J.htm

Data obtained from the respondent-level interview and questionnaire on physical activity.

In [188]:
activity

,PAQ605,PAQ610,PAD615,PAQ620,PAQ625,PAD630,PAQ635,PAQ640,PAD645,PAQ650,PAQ655,PAD660,PAQ665,PAQ670,PAD675,PAD680
SEQN,,,,,,,,,,,,,,,,
93705,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,1.0,2.0,60.0,300.0
93706,2.0,NaN,NaN,2.0,NaN,NaN,1.0,5.0,45.0,2.0,NaN,NaN,1.0,2.0,30.0,240.0
93708,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,1.0,5.0,30.0,120.0
93709,2.0,NaN,NaN,1.0,2.0,180.0,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,600.0
93711,2.0,NaN,NaN,2.0,NaN,NaN,1.0,5.0,60.0,1.0,4.0,60.0,1.0,2.0,30.0,420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102950,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,60.0
102952,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,1.0,6.0,60.0,120.0
102953,1.0,3.0,240.0,1.0,3.0,240.0,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,360.0


In [189]:
activity.columns

Index(['PAQ605', 'PAQ610', 'PAD615', 'PAQ620', 'PAQ625', 'PAD630', 'PAQ635',
       'PAQ640', 'PAD645', 'PAQ650', 'PAQ655', 'PAD660', 'PAQ665', 'PAQ670',
       'PAD675', 'PAD680'],
      dtype='object')

In [190]:
keepcols_activity = ['PAD615', 'PAQ610', 'PAD630', 'PAQ625', 'PAQ640', 'PAD645', 'PAD660', 
                     'PAQ655', 'PAD675', 'PAQ670', 'PAD680']
mapper_activity = {'PAD615':'work_vigorous_minperday', 
                   'PAQ610': 'work_vigorous_daysperweek',
                   'PAD630': 'work_moderate_minperday', 
                   'PAQ625': 'work_moderate_daysperweek',
                   'PAD645': 'transportation_minperday', 
                   'PAQ640': 'transportation_daysperweek',
                   'PAD660': 'recreation_vigorous_minperday', 
                   'PAQ655': 'recreation_vigorous_daysperweek',
                   'PAD675': 'recreation_moderate_minperday', 
                   'PAQ670': 'recreation_moderate_daysperweek',
                   'PAD680': 'sedentary_minsperday'}

In [191]:
keep_activity = activity[keepcols_activity].rename(mapper=mapper_activity, axis=1).fillna(0)
keep_activity

,work_vigorous_minperday,work_vigorous_daysperweek,work_moderate_minperday,work_moderate_daysperweek,transportation_daysperweek,transportation_minperday,recreation_vigorous_minperday,recreation_vigorous_daysperweek,recreation_moderate_minperday,recreation_moderate_daysperweek,sedentary_minsperday
SEQN,,,,,,,,,,,
93705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,2.0,300.0
93706,0.0,0.0,0.0,0.0,5.0,45.0,0.0,0.0,30.0,2.0,240.0
93708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,5.0,120.0
93709,0.0,0.0,180.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0
93711,0.0,0.0,0.0,0.0,5.0,60.0,60.0,4.0,30.0,2.0,420.0
...,...,...,...,...,...,...,...,...,...,...,...
102950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0
102952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,6.0,120.0
102953,240.0,3.0,240.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,360.0


In [192]:
keep_activity['vigorous_activity_minsperweek'] =\
            (keep_activity['work_vigorous_daysperweek']*keep_activity['work_vigorous_minperday']) +\
            (keep_activity['recreation_vigorous_daysperweek']*keep_activity['recreation_vigorous_daysperweek'])

keep_activity['moderate_activity_minsperweek'] =\
            (keep_activity['work_moderate_daysperweek']*keep_activity['work_moderate_minperday']) +\
            (keep_activity['recreation_moderate_daysperweek']*keep_activity['recreation_moderate_minperday']) +\
            (keep_activity['transportation_daysperweek']*keep_activity['transportation_minperday'])

keep_activity

,work_vigorous_minperday,work_vigorous_daysperweek,work_moderate_minperday,work_moderate_daysperweek,transportation_daysperweek,transportation_minperday,recreation_vigorous_minperday,recreation_vigorous_daysperweek,recreation_moderate_minperday,recreation_moderate_daysperweek,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek
SEQN,,,,,,,,,,,,,
93705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,2.0,300.0,0.0,120.0
93706,0.0,0.0,0.0,0.0,5.0,45.0,0.0,0.0,30.0,2.0,240.0,0.0,285.0
93708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,5.0,120.0,0.0,150.0
93709,0.0,0.0,180.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0,0.0,360.0
93711,0.0,0.0,0.0,0.0,5.0,60.0,60.0,4.0,30.0,2.0,420.0,16.0,360.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0
102952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,6.0,120.0,0.0,360.0
102953,240.0,3.0,240.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,360.0,720.0,720.0


In [193]:
keep_activity2 = keep_activity[['sedentary_minsperday',
                                'vigorous_activity_minsperweek',
                                'moderate_activity_minsperweek']]
keep_activity2

,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek
SEQN,,,
93705,300.0,0.0,120.0
93706,240.0,0.0,285.0
93708,120.0,0.0,150.0
93709,600.0,0.0,360.0
93711,420.0,16.0,360.0
...,...,...,...
102950,60.0,0.0,0.0
102952,120.0,0.0,360.0
102953,360.0,720.0,720.0


#### Smoking

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/SMQ_J.htm#Component_Description

History of cigarette use per respondent, self-reported during interviews.

In [194]:
smoking

,SMQ020,SMD030,SMQ040,SMQ050Q,SMQ050U,SMD057,SMQ078,SMD641,SMD650,SMD093,SMDUPCA,SMD100BR,SMD100FL,SMD100MN,SMD100LN,SMD100TR,SMD100NI,SMD100CO,SMQ621,SMD630,SMQ661,SMQ665A,SMQ665B,SMQ665C,SMQ665D,SMQ670,SMQ848,SMQ852Q,SMQ852U,SMQ890,SMQ895,SMQ900,SMQ905,SMQ910,SMQ915,SMAQUEX2
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93705,1.0,16.0,3.0,30.0,4.0,5.0,NaN,NaN,NaN,NaN,b'',b'',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,2.0,NaN,1.0
93706,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b'',b'',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,2.0,NaN,1.0
93707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b'',b'',NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
93708,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b'',b'',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,2.0,NaN,1.0
93709,1.0,15.0,1.0,NaN,NaN,NaN,1.0,30.0,5.0,1.0,b'* NO MATCH *',b'WAVE GREEN',1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,5.397605e-79,1.0,5.397605e-79,1.0,5.397605e-79,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102952,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b'',b'',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,2.0,NaN,1.0
102953,1.0,20.0,3.0,9.0,3.0,1.0,NaN,NaN,NaN,NaN,b'',b'',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.397605e-79,1.0,5.397605e-79,1.0,5.397605e-79,1.0
102954,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b'',b'',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,2.0,NaN,1.0


In [195]:
keepcols_smoking = ['SMQ020', 'SMQ040', 'SMQ905', 'SMQ915']
mapper_smoking = {'SMQ020': 'cigarettes_100+ in lifetime',
                 'SMQ040': 'current_cigarette_smoker',
                 'SMQ905': 'ecig_lastmonth',
                 'SMQ915': 'smokeless_lastmonth'}

In [198]:
keep_smoking = smoking[keepcols_smoking].rename(mapper=mapper_smoking, axis=1)
keep_smoking.round(1)

,cigarettes_100+ in lifetime,current_cigarette_smoker,ecig_lastmonth,smokeless_lastmonth
SEQN,,,,
93705,1.0,3.0,NaN,NaN
93706,2.0,NaN,NaN,NaN
93707,NaN,NaN,NaN,NaN
93708,2.0,NaN,NaN,NaN
93709,1.0,1.0,0.0,0.0
...,...,...,...,...
102952,2.0,NaN,NaN,NaN
102953,1.0,3.0,0.0,0.0
102954,2.0,NaN,NaN,NaN


## Preprocessing

NOTES TO SELF:
* Focus on adults; remove ages below 18 from dataset
* Remove pregnant women from dataset as their measurements may be skewed

In [197]:
dfs = [keep_demographic, keep_insurance, keep_measures, keep_bp2, a1c, cholesterol, 
       keep_insulin, keep_activity2, keep_smoking]

In [200]:
col_count = 0
for df in dfs:
    col_count += len(list(df.columns))
col_count

47